In [1]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from fancyimpute import mice
from fancyimpute import KNN, NuclearNormMinimization, SoftImpute, IterativeImputer, BiScaler
from sklearn.preprocessing import Imputer
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV
import math
import sklearn.metrics as sklm
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.decomposition import PCA

C:\Tools\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df_train_x = pd.read_csv('../data/train_new.csv')
df_train_x.shape

(1401, 18)

# PCA

In [4]:
def pca_reduce(df):
    pca = PCA(n_components=10)
    pca = pca.fit(df)
    
    unit_vec = pca.components_ #得到投影之單位向量
    print('單位向量:', unit_vec)
    
    reduced_data = pca.transform(df)
    print('降維後資料:', reduced_data.shape)
    
    return reduced_data


pca_reduce(df_train_x)

單位向量: [[ 5.51492847e-02 -3.93335663e-05  4.25221111e-06 -1.08029084e-03
   9.98477082e-01 -1.09348632e-04  1.31610589e-05  1.55733812e-04
   5.26803728e-04 -3.64932440e-04  4.12175475e-04 -2.09872086e-04
   2.68384750e-04 -3.38395170e-04  2.20453207e-05  6.81199528e-06
  -2.38799118e-04]
 [ 9.98464372e-01 -9.22613053e-05  2.98059744e-05  4.90115055e-03
  -5.51449847e-02 -3.89751287e-04  5.13156822e-04  6.77229523e-04
   7.62933441e-04 -3.37624770e-04  9.82563117e-04 -5.13573214e-04
   4.58176292e-04 -6.44032730e-04  2.32285908e-05  1.27354225e-06
  -6.87969654e-04]
 [-4.97526306e-03 -2.91682438e-03 -2.42473243e-03  9.96127246e-01
   1.29133198e-03 -1.50298440e-02  5.11571434e-02  2.23935478e-02
   1.66046119e-02 -2.05392387e-02  2.96808479e-02 -2.95571914e-02
   1.40326154e-02 -2.85808982e-02  7.41082374e-04  4.92264554e-04
  -2.96404044e-02]
 [ 1.50751571e-03  3.13584941e-02  2.31088535e-02  6.38923888e-02
   9.25769847e-04  1.80849521e-01 -5.69655090e-04 -2.29925856e-01
  -5.60263665

array([[-7.95753635e+03, -1.44933554e+03, -1.57379211e+02, ...,
         2.74296830e+00, -1.03018915e+01, -1.81933576e+00],
       [-7.23012789e+03,  1.61190529e+03,  3.42680385e+01, ...,
        -4.00443039e+00,  1.27568875e+01,  5.72560273e+00],
       [ 6.59996936e+03, -1.60580419e+03, -8.67423416e+01, ...,
        -1.24697817e+01, -7.21150876e+00, -1.61846486e+01],
       ...,
       [-5.95134230e+03, -3.65695768e+02,  2.66868309e+02, ...,
        -7.19767826e+00, -1.64920861e+00,  9.19929962e+00],
       [ 8.25170990e+03, -1.66959996e+03, -1.05724081e+02, ...,
        -7.30516921e+00, -6.76745557e+00, -7.95007100e+00],
       [-9.34356504e+03, -1.68259231e+03, -1.55813296e+02, ...,
         6.59182185e+00,  1.73823850e+01, -4.23987826e+00]])

--- 
# Modeling

### random forest

In [ ]:

def rf_modeling(trainX, trainY):
    rf = RandomForestRegressor()
    param_grid = { 
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
    }

    grid_rf = GridSearchCV(rf, param_grid, n_jobs=-1, cv=5)
    forest_model = grid_rf.fit(trainX, trainY)
    
    return forest_model


forest_model = rf_modeling(trainX, trainY)
y_pred = forest_model.predict(testX)

print('Root Mean Square Error = ' + str(math.sqrt(sklm.mean_squared_error(testY, y_pred))))

### XGBoost

In [ ]:

def xgb_modeling(trainX, trainY):
    xgb_reg = XGBRegressor(n_estimators=100, max_depth=10,learning_rate=0.01,random_state=100,subsample=0.8)
    params = {'min_child_weight':[4,5], 'gamma':[i/10.0 for i in range(3,6)],  'subsample':[i/10.0 for i in range(6,11)],
              'colsample_bytree':[i/10.0 for i in range(6,11)], 'max_depth': [2,3,4]}
    grid = GridSearchCV(xgb_reg, params,scoring='neg_mean_squared_error',cv=5)
    
    xgb_model = grid.fit(trainX, trainY)
    return xgb_model


xgb_model = xgb_modeling(trainX, trainY)
y_pred = xgb_model.predict(testX)

print('Root Mean Square Error = ' + str(math.sqrt(sklm.mean_squared_error(testY, y_pred))))